<a href="https://colab.research.google.com/github/joker2017/Calculator/blob/master/mobilnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sys
sys.path.append("https://github.com/tensorflow/models/blob/master/research/slim")

In [0]:
import tensorflow as tf
import os

saver = tf.train.import_meta_graph(os.path.join(model_path,r"resnet_v2/model.ckpt-258931.meta"))
#import_meta_graph может загружать структуру диаграммы непосредственно из файла meta

with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    # latest_checkpoint проверяет файл контрольной точки, чтобы найти последнюю модель
    # восстановление веса диаграммы восстановления
    saver.restore(sess,tf.train.latest_checkpoint(r"./model/resnet_v2"))
    graph = sess.graph
    # get_tensor_by_name получает тензор по имени тензора
    print(sess.run(graph.get_tensor_by_name("resnet_model/conv2d/kernel:0")))

In [0]:
# Временно добавьте slim в путь поиска python
import sys
sys.path.append("./models/research/slim")

# Импортировать mobilenet_v2
from nets.mobilenet import mobilenet_v2
#Сбросить карту
tf.reset_default_graph()

# Импортировать мобильную сеть, сначала построить структуру графа.
#Loading ’‘ После загрузки tf.get_default_graph () содержит структуру графика расчета мобильной сети.
#Вы можете использовать tf.get_collection (tf.GraphKeys.TRAINABLE_VARIABLES), чтобы сравнить разницу до и после reset_graph

images = tf.placeholder(tf.float32,(None,224,224,3))
with tf.contrib.slim.arg_scope(mobilenet_v2.training_scope(is_training=False)):
    logits, endpoints = mobilenet_v2.mobilenet(images,depth_multiplier=1.4)

# Определите класс заставки для восстановления веса графика
saver = tf.train.Saver()
with tf.Session() as sess:
    # Latest_checkpoint проверяет файл контрольной точки контрольной точки, чтобы найти последнюю модель
    # Восстановить восстановить вес графика
    saver.restore(sess,tf.train.latest_checkpoint("./model_ckpt/mobilenet_v2"))
    # Get_tensor_by_name получает тензор по имени тензора
    print(sess.run(tf.get_default_graph().get_tensor_by_name("MobilenetV2/Conv/weights:0")).shape)

In [0]:

# Прочитайте сохраненный файл pb и проанализируйте его в соответствующем буфере протокола GraphDef.
gd =  tf.GraphDef.FromString(open('./model_ckpt/mobilenet_v2/mobilenet_v2_1.4_224_frozen.pb',"rb").read())
# Import_graph_def загружает график, сохраненный в graphdef, в текущий граф, return_elements возвращает указанный тензор
inp, predictions = \
tf.import_graph_def(gd,return_elements=["input:0","MobilenetV2/Predictions/Reshape_1:0"])

# График расчета в это время может быть использован непосредственно для прогнозирования
# Вытащить картинку !wget https://upload.wikimedia.org/wikipedia/commons/f/fe/Giant_Panda_in_Beijing_Zoo_1.JPG -O panda.jpg
from PIL import Image
img = np.array(Image.open('panda.jpg').resize((224, 224))).astype(np.float) / 128 - 1
# Inp - это входные данные, которые необходимо ввести, а предсказания - это прогнозирующая структура, которую необходимо выводить.
with tf.Session(graph=inp.graph) as sess:
    x = sess.run(predictions,feed_dict={inp:img.reshape(1,224,224,3)})

In [0]:
tf.reset_default_graph()
# Построить расчетную диаграмму
images = tf.placeholder(tf.float32,(None,224,224,3))
with tf.contrib.slim.arg_scope(mobilenet_v2.training_scope(is_training=False)):
    logits, endpoints = mobilenet_v2.mobilenet(images,depth_multiplier=1.4)

# Получить целевой тензор, добавить новый слой
with tf.variable_scope("finetune_layers"):
    # Получить целевой тензор и вынуть тензор указанного слоя в мобильной сети
    mobilenet_tensor = tf.get_default_graph().get_tensor_by_name("MobilenetV2/expanded_conv_14/output:0")
    # Передаем тензор на новый слой
    x = tf.layers.Conv2D(filters=256,kernel_size=3,name="conv2d_1")(mobilenet_tensor)
    # Наблюдаем, обновляется ли вес нового слоя tf.summary.histogram ("conv2d_1", x)
    x = tf.nn.relu(x,name="relu_1")
    x = tf.layers.Conv2D(filters=256,kernel_size=3,name="conv2d_2")(x)
    x = tf.layers.Conv2D(10,3,name="conv2d_3")(x)
    predictions = tf.reshape(x, (-1,10))

In [0]:
# горячее кодирование
def to_categorical(data, nums):
    return np.eye(nums)[data]
# Произвольно генерировать данные
x_train = np.random.random(size=(141,224,224,3))
y_train = to_categorical(label_fake,10)

#Конфигурация условий обучения
## метка-заполнитель
y_label = tf.placeholder(tf.int32, (None,10))
## Собирайте переменные в области видимости finetune_layers, обновляйте только вес добавленного слоя
train_var = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope="finetune_layers")
##Потеря определения
loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_label,logits=predictions)
## Определите метод оптимизации, используйте var_list, чтобы указать вес для обновления, и обновляйте только вес train_var в это время.
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss,var_list=train_var)
##Наблюдайте, обновляется ли вес нового слоя
tf.summary.histogram("mobilenet_conv8",tf.get_default_graph().get_tensor_by_name('MobilenetV2/expanded_conv_8/depthwise/depthwise_weights:0'))
tf.summary.histogram("mobilenet_conv9",tf.get_default_graph().get_tensor_by_name('MobilenetV2/expanded_conv_9/depthwise/depthwise_weights:0'))

##Объединить все резюме
merge_all = tf.summary.merge_all()

## Установите количество итераций и количество университетов
epochs = 10
batch_size = 16

# Получить функцию из указанного списка переменных var_list
def get_var_list(target_tensor=None):
#Получить функцию из указанного списка переменных var_list
    if target_tensor==None:
        target_tensor = r"MobilenetV2/expanded_conv_14/output:0"
    target = target_tensor.split("/")[1]
    all_list = []
    all_var = []
    # пройти все переменные, имя узла получает имя переменной
    # Не используйте tf.trainable_variables (), потому что moving_mean / дисперсия batchnorm не принадлежит к обучаемой переменной
    for var in tf.global_variables():
        if var != []:
            all_list.append(var.name)
            all_var.append(var)
    try:
        all_list = list(map(lambda x:x.split("/")[1],all_list))
        # Найти индекс соответствующей переменной области видимости
        ind = all_list[::-1].index(target)
        ind = len(all_list) -  ind - 1
        print(ind)
        del all_list
        return all_var[:ind+1]
    except:
        print("target_tensor is not exist!")

# Имя целевого тензора, чтобы получить список переменных var_list, который должен загружать веса из файла
target_tensor = "MobilenetV2/expanded_conv_14/output:0"
var_list = get_var_list(target_tensor)
saver = tf.train.Saver(var_list=var_list)

# Загрузите веса в файл и обучите новый слой
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    writer = tf.summary.FileWriter(r"./logs", sess.graph)
    ## Параметры инициализации: загрузка весов из файлов train_var использует функции инициализации
    sess.run(tf.variables_initializer(var_list=train_var))
    saver.restore(sess,tf.train.latest_checkpoint("./model_ckpt/mobilenet_v2"))
    
    for i in range(2000):
        start = (i*batch_size) % x_train.shape[0]
        end = min(start+batch_size, x_train.shape[0])
        _, merge, losses = sess.run([train_step,merge_all,loss],\
                             feed_dict={images:x_train[start:end],\
                                        y_label:y_train[start:end]})
        if i%100==0:
           writer.add_summary(merge, i)